In [1]:
import pandas as pd
import numpy as np
import Clean_data as cld
import Transform_Data as tfd
import Visualize_Data as vld

In [2]:
df = pd.read_csv('Datasets/Discoveries/Degree_to_job_title_value_counts.csv')


In [3]:
df.columns

Index(['Unnamed: 0', 'major', 'category', 'title', 'count'], dtype='object')

### Establish Core of Education Lookup Table

In [4]:

lookup_df = df[['major', 'category']]

In [5]:
lookup_df.head(10)

,major,category
0,3d modelling,technology
1,3d modelling,technology
2,3d modelling,technology
3,3d modelling,technology
4,3d modelling,technology
5,3d modelling,technology
6,3d modelling,technology
7,3d modelling,technology
8,3d modelling,technology
9,3d modelling,technology


### Add other tables that can tie into the core table

In [6]:
st_degree_ct_df = pd.read_csv('Datasets/Discoveries/State_Degree_Counts.csv')
global_indst_df = pd.read_csv('Datasets/Discoveries/Industry_counts_global.csv')


degree_to_position_ct = pd.read_csv('Datasets/Discoveries/Degree_to_job_title_total_counts.csv')
degree_specs_placement= pd.read_csv('Datasets/Discoveries/Degree_Specs_Pruned_Placement.csv')
degree_payback = pd.read_csv('Datasets/Discoveries/Degree_payback_grouped_mean.csv')

### Explore and assess usefullness

In [7]:
st_degree_ct_df.head(3)

,State,Bachelor's Degree Holders,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",sci_eng,sci_eng_rel,business,education,art_hum_oth
0,District of Columbia,602858.0,292418.0,31224.0,78228.0,20502.0,180486.0,0.485053,0.051793,0.129762,0.034008,0.299384
1,Washington,3911264.0,1647784.0,333010.0,626424.0,376178.0,927868.0,0.421292,0.085141,0.160159,0.096178,0.237230
2,California,18856968.0,7859122.0,1562792.0,3442188.0,1160732.0,4832134.0,0.416775,0.082876,0.182542,0.061555,0.256252


Can link states with ratios of overarching degree-types, and assign a weight, per degree type, per region

In [8]:
global_indst_df.head(3)

,industry,Count,related_field
0,information technology and services,534948,"Science, Technology, Engineering"
1,marketing and advertising,385649,"Business, Admin, PR"
2,real estate,274165,"Business, Admin, PR"


In [9]:
global_indst_df.columns = ['industry', 'count', 'field']

Can group each industry into a broader 'related degree' as well as 'industry type' pool, Will require a little more manual work on my part

In [10]:
degree_to_position_ct.head(3)

,Unnamed: 0,major,category,count
0,0,3d modelling,technology,299.0
1,1,3d modelling and animation,technology,52.0
2,2,accountancy,finance,354609.0


In [11]:
degree_to_position_ct.groupby('category')['major'].count()

category
admin          58
finance        34
science        67
technology    108
Name: major, dtype: int64

In [12]:
degree_to_position_ct.groupby('category')['count'].sum()

category
admin         22312579.0
finance       17856019.0
science        5985411.0
technology    12396555.0
Name: count, dtype: float64

I've realized after some further diving that this dataset it too limited to provide meaningful insight.

In [13]:
degree_specs_placement.head(3)

,early_specialisation,degree,specialisation,mean,count,median,std,min,max
0,Science,Sci&Tech,Mkt&Fin,322720.0000,25,275000,115811.45600,200000,690000
1,Science,Sci&Tech,Mkt&HR,304066.6667,15,276000,72335.00108,210000,450000
2,Commerce,Comm&Mgmt,Mkt&Fin,299685.1852,54,277500,116492.74400,200000,940000


In [14]:
degree_specs_placement.shape

(6, 9)

based on small sample-size, this on may be cut for this particular phase

In [15]:
degree_payback.head(3)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary
0,Accounting,46000.0,77100.0,0.676
1,Aerospace Engineering,57700.0,101000.0,0.750
2,Agriculture,42600.0,71900.0,0.688


This will pair up nicely for the weight portion of my project. saved for later. 

### Using the survivng datasets

In [16]:
global_indst_df.head(3)

,industry,count,field
0,information technology and services,534948,"Science, Technology, Engineering"
1,marketing and advertising,385649,"Business, Admin, PR"
2,real estate,274165,"Business, Admin, PR"


I've created 5 buckets to fit my education and industries into.

* A: Business, Admin, PR
* B: Architecture, Design
* C: Consumer, Retail, Arts, Humanities
* D: Health, Education, Government, Civil
* E: Science, Technology, Engineering

#####  State Degree Counts

In [17]:
st_degree_ct_df.head(3)

,State,Bachelor's Degree Holders,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",sci_eng,sci_eng_rel,business,education,art_hum_oth
0,District of Columbia,602858.0,292418.0,31224.0,78228.0,20502.0,180486.0,0.485053,0.051793,0.129762,0.034008,0.299384
1,Washington,3911264.0,1647784.0,333010.0,626424.0,376178.0,927868.0,0.421292,0.085141,0.160159,0.096178,0.237230
2,California,18856968.0,7859122.0,1562792.0,3442188.0,1160732.0,4832134.0,0.416775,0.082876,0.182542,0.061555,0.256252


In [18]:
st_degree_ct_df = st_degree_ct_df[['State', 'Bachelor\'s Degree Holders', 'sci_eng', 'sci_eng_rel', 'business',
       'education', 'art_hum_oth']]

Step one is to remove the raw counts, aside from over-all bachelors degree holders. this will reduce the weight of pitting larger and smaller states against each other.

In [19]:
pd.options.display.float_format = '{:20,.2f}'.format
melted_st_deg_ct = st_degree_ct_df.melt(id_vars=['State'])

In [20]:
melted_st_deg_ct = melted_st_deg_ct.reset_index(drop=True)
melted_st_deg_ct.columns = ['state', 'field', 'ratio']

In [21]:
melted_st_deg_ct

,state,field,ratio
0,District of Columbia,Bachelor's Degree Holders,"602,858.00"
1,Washington,Bachelor's Degree Holders,"3,911,264.00"
2,California,Bachelor's Degree Holders,"18,856,968.00"
3,Maryland,Bachelor's Degree Holders,"3,420,460.00"
4,Massachusetts,Bachelor's Degree Holders,"4,363,486.00"
...,...,...,...
301,Louisiana,art_hum_oth,0.23
302,Kentucky,art_hum_oth,0.24
303,Nebraska,art_hum_oth,0.22
304,North Dakota,art_hum_oth,0.18


In [22]:
degree_to_position_ct.head(3)

,Unnamed: 0,major,category,count
0,0,3d modelling,technology,299.00
1,1,3d modelling and animation,technology,52.00
2,2,accountancy,finance,"354,609.00"


In [23]:
for i in range(len(degree_to_position_ct)):
    val = degree_to_position_ct.loc[i, 'category']
    if(val == 'science'):
        degree_to_position_ct.loc[i, 'category'] =  'Science, Technology, Engineering' 
    elif(val == 'technology'):
        degree_to_position_ct.loc[i, 'category'] =  'Science, Technology, Engineering'
    elif(val == 'finance'):
        degree_to_position_ct.loc[i, 'category'] =  'Business, Admin, PR'
    elif(val == 'admin'):
        degree_to_position_ct.loc[i, 'category'] =  'Business, Admin, PR'
    else:
         degree_to_position_ct.loc[i, 'category'] = 'Unassigned'

In [24]:
for i in range(len(melted_st_deg_ct)):
    val = melted_st_deg_ct.loc[i, 'field']
    if(val == 'Bachelor\'s Degree Holders'):
        melted_st_deg_ct.loc[i, 'field'] =  'State_Total' 
    elif(val == 'sci_eng'):
        melted_st_deg_ct.loc[i, 'field'] =  'Science, Technology, Engineering'
    elif(val == 'sci_eng_rel'):
        melted_st_deg_ct.loc[i, 'field'] =  'Architecture, Design'
    elif(val == 'business'):
        melted_st_deg_ct.loc[i, 'field'] =  'Business, Admin, PR'
    elif(val == 'education'):
        melted_st_deg_ct.loc[i, 'field'] =  'Health, Education, Government, Civil'
    elif(val == 'art_hum_oth'):
        melted_st_deg_ct.loc[i, 'field'] =  'Consumer, Retail, Arts, Humanities'
    else:
        melted_st_deg_ct.loc[i, 'field'] = 'Unassigned'

In [25]:
melted_st_deg_ct.to_csv('Final_Data/Melted_State_Degree_Ratios.csv')
degree_to_position_ct.to_csv('Final_Data/Degree_Position_Count.csv') #Keep but not using
global_indst_df.to_csv('Final_Data/Industry_counts_global.csv')